In [17]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dropout, GRU, LSTM
import pywt
import cufflinks as cf
cf.go_offline()

In [2]:
class AttentionMechanism(Layer):
    def __init__(self, return_sequences=True):
        super(AttentionMechanism, self).__init__()
        self.return_sequences = return_sequences

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")

    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x,self.W)+self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x*a

        if self.return_sequences:
            return output

        return tf.reduce_sum(output, axis=1)

In [3]:
class PreprocessData:
    def __init__(self, file_path, feature_column, look_back=2):
        self.file_path = file_path
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.data_scaled = None
        self.X = None
        self.y = None

    def check_look_back(self):
        if self.look_back == 0:
            raise ValueError('You can not use number 0')

    def load_preprocess_data(self):
        df = pd.read_csv(self.file_path, index_col=['date'], parse_dates=True)
        df[self.feature_column].fillna(method='ffill', inplace=True)
        self.data_scaled = self.scaler.fit_transform(df[[self.feature_column]])

    def create_dataset(self):
        X, y = [], []
        
        for i in range(len(self.data_scaled) - self.look_back):
            a = self.data_scaled[i:(i + self.look_back), 0]
            X.append(a)
            y.append(self.data_scaled[i + self.look_back, 0])
        
        self.X, self.y = np.array(X), np.array(y)

        X_dif = self.X[:-1] - self.X[1:]
        X_new = np.hstack([self.X[:-1], X_dif])
        self.y = self.y[:-1]

        self.X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))

    def split_data(self, train_size=0.8):
        split_idx = int(len(self.X) * train_size)
        X_train, X_test = self.X[:split_idx], self.X[split_idx:]
        y_train, y_test = self.y[:split_idx], self.y[split_idx:]
        
        return X_train, X_test, y_train, y_test

In [4]:
class TrainModel:
    def __init__(self):
        self.model = None

    def build_model_simple_rnn(self, input_shape):
        self.model = Sequential([
            SimpleRNN(62, activation='relu', input_shape=input_shape, return_sequences=True),
            SimpleRNN(29, activation='relu', input_shape=input_shape, return_sequences=True),
            AttentionMechanism(return_sequences=True),
            SimpleRNN(15, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

        return self.model
    
    def build_model_gru(self, input_shape):
        self.model = Sequential([
            GRU(94, activation='relu', input_shape=input_shape, return_sequences=True),
            GRU(100, activation='relu', return_sequences=True),
            Dropout(0.2),
            AttentionMechanism(return_sequences=True),
            GRU(14, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

        return self.model
    
    def build_model_lstm(self, input_shape):
        self.model = Sequential([
            LSTM(90, activation='relu', input_shape=input_shape, return_sequences=True),
            LSTM(79, activation='relu', return_sequences=True),
            Dropout(0.2),
            AttentionMechanism(return_sequences=True),
            LSTM(30, activation='relu'),
            Dense(1)
        ])
        self.model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

        return self.model

    def train_model(self, X_train, y_train, epochs=100, batch_size=64):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    def evaluate_model(self, X_test, y_test, scaler):
        predictions = self.model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)
        y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
        
        mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100
        mae = np.mean(np.abs(y_test_actual - predictions))
        
        print(f"MAPE: {mape}%")
        print(f"MAE: {mae}")
        
        return predictions, y_test_actual

    def plot_results(self, predictions, y_test_actual, feature_column):
        df = pd.DataFrame({
            'Actual': y_test_actual.ravel(),
            'Predicted': predictions.ravel()
        })
        
        df.iplot(title=f"Прогноз vs Реальные значения (Тест) для {feature_column}",
                 xTitle="Время",
                 yTitle="Значение",
                 theme="solar")

In [6]:
class ValidationData:
    def __init__(self, feature_column, look_back=1):
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.model = None
        self.X_val = None
        self.y_val = None
        self.predictions_val = None
        self.y_val_actual = None
        self.model = TrainModel()

    def check_look_back(self):
        if self.look_back == 0:
            raise ValueError('You can not use number 0')

    def load_validation_data(self, file_path, model):
        try:
            df = pd.read_csv(file_path, index_col=['date'], parse_dates=True)
            df[self.feature_column].fillna(method='ffill', inplace=True)
            self.data_scaled = self.scaler.fit_transform(df[[self.feature_column]])

            X, y = [], []
            
            for i in range(len(self.data_scaled) - self.look_back):
                a = self.data_scaled[i:(i + self.look_back), 0]
                X.append(a)
                y.append(self.data_scaled[i + self.look_back, 0])
            
            self.X, self.y = np.array(X), np.array(y)

            X_dif = self.X[:-1] - self.X[1:]
            X_new = np.hstack([self.X[:-1], X_dif])
            self.y = self.y[:-1]

            self.X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
    
            predictions = model.predict(self.X)
            predictions = self.scaler.inverse_transform(predictions)
            y_actual = self.scaler.inverse_transform(self.y.reshape(-1, 1))

            return predictions, y_actual
            
        except Exception as e:
            print(f"Error in load_validation_data: {str(e)}")

    def plot_val_results(self, predictions, y_actual):
        if predictions is not None and y_actual is not None:
            df_val = pd.DataFrame({
                'Actual (Validation)': y_actual.ravel(),
                'Predicted (Validation)': predictions.ravel()
            })
            df_val.iplot(title=f"Прогноз vs Реальные значения (Валидация) для {self.feature_column}",
                         xTitle="Время",
                         yTitle="Значение",
                         theme="solar")
        else:
            print('Валидационные данные не введены.')

In [6]:
# Преобразование данных для обучения модели
preprocess_meantemp = PreprocessData('DailyDelhiClimateTrain.csv', 'meantemp', look_back=5)
preprocess_meantemp.check_look_back()
preprocess_meantemp.load_preprocess_data()
preprocess_meantemp.create_dataset()
X_train_meantemp, X_test_meantemp, y_train_meantemp, y_test_meantemp = preprocess_meantemp.split_data()

preprocess_humidity = PreprocessData('DailyDelhiClimateTrain.csv', 'humidity', look_back=5)
preprocess_humidity.check_look_back()
preprocess_humidity.load_preprocess_data()
preprocess_humidity.create_dataset()
X_train_humidity, X_test_humidity, y_train_humidity, y_test_humidity = preprocess_humidity.split_data()

preprocess_wind_speed = PreprocessData('DailyDelhiClimateTrain.csv', 'wind_speed', look_back=5)
preprocess_wind_speed.check_look_back()
preprocess_wind_speed.load_preprocess_data()
preprocess_wind_speed.create_dataset()
X_train_wind_speed, X_test_wind_speed, y_train_wind_speed, y_test_wind_speed = preprocess_wind_speed.split_data()

preprocess_meanpressure = PreprocessData('DailyDelhiClimateTrain.csv', 'meanpressure', look_back=4)
preprocess_meanpressure.check_look_back()
preprocess_meanpressure.load_preprocess_data()
preprocess_meanpressure.create_dataset()
X_train_meanpressure, X_test_meanpressure, y_train_meanpressure, y_test_meanpressure = preprocess_meanpressure.split_data()

In [7]:
# Тренировка модели
trainer_meantemp = TrainModel()
trainer_meantemp.build_model_simple_rnn(input_shape=(X_train_meantemp.shape[1], 1))
trainer_meantemp.train_model(X_train_meantemp, y_train_meantemp)

trainer_humidity = TrainModel()
trainer_humidity.build_model_simple_rnn(input_shape=(X_train_humidity.shape[1], 1))
trainer_humidity.train_model(X_train_humidity, y_train_humidity)

trainer_wind_speed = TrainModel()
trainer_wind_speed.build_model_simple_rnn(input_shape=(X_train_wind_speed.shape[1], 1))
trainer_wind_speed.train_model(X_train_wind_speed, y_train_wind_speed)

trainer_meanpressure = TrainModel()
trainer_meanpressure.build_model_simple_rnn(input_shape=(X_train_meanpressure.shape[1], 1))
trainer_meanpressure.train_model(X_train_meanpressure, y_train_meanpressure)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.1627 - val_loss: 0.0150
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0143 - val_loss: 0.0036
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0047 - val_loss: 0.0024
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0046 - val_loss: 0.0104
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0077 - val_loss: 0.0020
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0043 - val_loss: 0.0021
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0029 - val_loss: 0.0011
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0021 - val_loss: 8.0740e-04
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.3577e-04 - val_loss: 3.0779e-04
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.5677e-04 - val_loss: 2.0469e-04
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s

In [8]:
# Визуализация результатов обучения модели
predictions_meantemp, y_test_actual_meantemp = trainer_meantemp.evaluate_model(X_test_meantemp, y_test_meantemp, preprocess_meantemp.scaler)
trainer_meantemp.plot_results(predictions_meantemp, y_test_actual_meantemp, preprocess_meantemp.feature_column)

predictions_humidity, y_test_actual_humidity = trainer_humidity.evaluate_model(X_test_humidity, y_test_humidity, preprocess_humidity.scaler)
trainer_humidity.plot_results(predictions_humidity, y_test_actual_humidity, preprocess_humidity.feature_column)

predictions_wind_speed, y_test_actual_wind_speed = trainer_wind_speed.evaluate_model(X_test_wind_speed, y_test_wind_speed, preprocess_wind_speed.scaler)
trainer_wind_speed.plot_results(predictions_wind_speed, y_test_actual_wind_speed, preprocess_wind_speed.feature_column)

predictions_meanpressure, y_test_actual_meanpressure = trainer_meanpressure.evaluate_model(X_test_meanpressure, y_test_meanpressure, preprocess_meanpressure.scaler)
trainer_meanpressure.plot_results(predictions_meanpressure, y_test_actual_meanpressure, preprocess_meanpressure.feature_column)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
MAPE: 0.3276297405881455%
MAE: 0.09628299522588331


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
MAPE: 1.6886056125395499%
MAE: 0.86325406819272


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
MAPE: 11.665385170847138%
MAE: 0.7950768335639486


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
MAPE: 144.0475605039483%
MAE: 42.451493903681325


In [9]:
validation_data_meantemp = ValidationData(preprocess_meantemp.feature_column, preprocess_meantemp.look_back)
predictions_val_meantemp, y_val_actual_meantemp = validation_data_meantemp.load_validation_data('DailyDelhiClimateTest.csv', trainer_meantemp.model)

validation_data_humidity = ValidationData(preprocess_humidity.feature_column, preprocess_humidity.look_back)
predictions_val_humidity, y_val_actual_humidity = validation_data_humidity.load_validation_data('DailyDelhiClimateTest.csv', trainer_humidity.model)

validation_data_wind_speed = ValidationData(preprocess_wind_speed.feature_column, preprocess_wind_speed.look_back)
predictions_val_wind_speed, y_val_actual_wind_speed = validation_data_wind_speed.load_validation_data('DailyDelhiClimateTest.csv', trainer_wind_speed.model)

validation_data_meanpressure = ValidationData(preprocess_meanpressure.feature_column, preprocess_meanpressure.look_back)
predictions_val_meanpressure, y_val_actual_meanpressure = validation_data_meanpressure.load_validation_data('DailyDelhiClimateTest.csv', trainer_meanpressure.model)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [10]:
# Визуализация валидационных данных
validation_data_meantemp.plot_val_results(predictions_val_meantemp, y_val_actual_meantemp)
validation_data_humidity.plot_val_results(predictions_val_humidity, y_val_actual_humidity)
validation_data_wind_speed.plot_val_results(predictions_val_wind_speed, y_val_actual_wind_speed)
validation_data_meanpressure.plot_val_results(predictions_val_meanpressure, y_val_actual_meanpressure)

In [10]:
import matplotlib.pyplot as plt

## Meanpressure

In [31]:
class PreprocessDataMeanPressure:
    def __init__(self, file_path, feature_column='meanpressure', look_back=2):
        self.file_path = file_path
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = None
        self.data_scaled = None
        self.X = None
        self.y = None

    def load_preprocess_data(self):
        df = pd.read_csv(self.file_path, index_col=['date'], parse_dates=True)
        df[self.feature_column] = df[self.feature_column].apply(lambda x: x - 997)
        df[self.feature_column].fillna(method='ffill', inplace=True)
        
        self.scaler = StandardScaler()
        self.data_scaled = self.scaler.fit_transform(df[[self.feature_column]])

    def check_look_back(self):
        if self.look_back == 0:
            raise ValueError('You can not use number 0')

    def create_dataset(self):
        X, y = [], []
        
        for i in range(len(self.data_scaled) - self.look_back):
            a = self.data_scaled[i:(i + self.look_back), 0]
            X.append(a)
            y.append(self.data_scaled[i + self.look_back, 0])
        
        self.X, self.y = np.array(X), np.array(y)

        X_dif = self.X[:-1] - self.X[1:]
        X_new = np.hstack([self.X[:-1], X_dif])
        self.y = self.y[:-1]

        self.X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))

    def split_data(self, train_size=0.8):
        split_idx = int(len(self.X) * train_size)
        X_train, X_test = self.X[:split_idx], self.X[split_idx:]
        y_train, y_test = self.y[:split_idx], self.y[split_idx:]
        
        X_test[0, 0, 0] = self.scaler.transform([[1000 - 997]])[0, 0]

        return X_train, X_test, y_train, y_test

In [39]:
class ValidationDataMeanPressure:
    def __init__(self, feature_column='meanpressure', look_back=2, scaler=None):
        self.feature_column = feature_column
        self.look_back = look_back
        self.scaler = scaler
        self.X = None
        self.y = None
        self.model = None

    def load_validation_data(self, file_path, model):
        self.model = model
        try:
            df = pd.read_csv(file_path, index_col=['date'], parse_dates=True)
            df.at[df.index[0], self.feature_column] = 1000
            df[self.feature_column] = df[self.feature_column].apply(lambda x: x - 997)
            df[self.feature_column].fillna(method='ffill', inplace=True)
            
            data_scaled = self.scaler.transform(df[[self.feature_column]])
            
            X, y = [], []
            for i in range(len(data_scaled) - self.look_back):
                a = data_scaled[i:(i + self.look_back), 0]
                X.append(a)
                y.append(data_scaled[i + self.look_back, 0])
            
            X, y = np.array(X), np.array(y)

            X_dif = X[:-1] - X[1:]
            X_new = np.hstack([X[:-1], X_dif])
            y = y[:-1]

            X = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))
        
            predictions = self.model.predict(X)
            predictions = self.scaler.inverse_transform(predictions)
            y_actual = self.scaler.inverse_transform(y.reshape(-1, 1))

            return predictions, y_actual
            
        except Exception as e:
            print(f"Error in load_validation_data: {str(e)}")
            return None, None
    
    def plot_val_results(self, predictions, y_actual):
        if predictions is not None and y_actual is not None:
            predictions = predictions + 997
            y_actual = y_actual + 997
            df_val = pd.DataFrame({
                'Actual (Validation)': y_actual.ravel(),
                'Predicted (Validation)': predictions.ravel()
            })
            df_val.iplot(title=f"Прогноз vs Реальные значения (Валидация) для {self.feature_column}",
                         xTitle="Время",
                         yTitle="Значение",
                         theme="solar")
        else:
            print('Валидационные данные не введены.')


In [40]:
preprocess_meanpressure = PreprocessDataMeanPressure('DailyDelhiClimateTrain.csv', 'meanpressure', look_back=4)
preprocess_meanpressure.check_look_back()
preprocess_meanpressure.load_preprocess_data()
preprocess_meanpressure.create_dataset()
X_train_meanpressure, X_test_meanpressure, y_train_meanpressure, y_test_meanpressure = preprocess_meanpressure.split_data()

trainer_meanpressure = TrainModel()
trainer_meanpressure.build_model_simple_rnn(input_shape=(X_train_meanpressure.shape[1], 1))
trainer_meanpressure.train_model(X_train_meanpressure, y_train_meanpressure)

predictions_meanpressure, y_test_actual_meanpressure = trainer_meanpressure.evaluate_model(X_test_meanpressure, y_test_meanpressure, preprocess_meanpressure.scaler)
trainer_meanpressure.plot_results(predictions_meanpressure, y_test_actual_meanpressure, preprocess_meanpressure.feature_column)

validation_meanpressure = ValidationDataMeanPressure('meanpressure', look_back=4, scaler=preprocess_meanpressure.scaler)
predictions_val_meanpressure, y_val_actual_meanpressure = validation_meanpressure.load_validation_data('DailyDelhiClimateTest.csv', trainer_meanpressure.model)
validation_meanpressure.plot_val_results(predictions_val_meanpressure, y_val_actual_meanpressure)


Epoch 1/100
15/15 [==============================] - 2s 31ms/step - loss: 0.0014 - val_loss: 8.6220e-04
Epoch 2/100
15/15 [==============================] - 0s 13ms/step - loss: 4.4894e-04 - val_loss: 1.7249e-04
Epoch 3/100
15/15 [==============================] - 0s 13ms/step - loss: 1.9830e-04 - val_loss: 1.1759e-04
Epoch 4/100
15/15 [==============================] - 0s 11ms/step - loss: 1.2825e-04 - val_loss: 7.9415e-05
Epoch 5/100
15/15 [==============================] - 0s 12ms/step - loss: 8.5227e-05 - val_loss: 5.7448e-05
Epoch 6/100
15/15 [==============================] - 0s 12ms/step - loss: 6.9447e-05 - val_loss: 6.7788e-05
Epoch 7/100
15/15 [==============================] - 0s 11ms/step - loss: 6.4428e-05 - val_loss: 4.3176e-05
Epoch 8/100
15/15 [==============================] - 0s 12ms/step - loss: 6.0489e-05 - val_loss: 3.5162e-05
Epoch 9/100
15/15 [==============================] - 0s 14ms/step - loss: 4.8340e-05 - val_loss: 4.1070e-05
Epoch 10/100
15/15 [============

4/4 [==============================] - 0s 9ms/step
